# KaelumAI Testing

## Setup

In [1]:
cd ..

/home/ash/projects/KaelumAI


In [ ]:
%pip install -e .

In [ ]:
%pip install sympy

In [2]:
from kaelum import enhance
import time

MODEL = "llama3.2:3b"

✅ Testing environment ready
Model: tinyllama
Config: temp=0.3, tokens=512, iter=1


## Test LLMs

In [9]:
# Test models
models = ["llama3.2:3b", "qwen2.5:7b"]
query = "What is 15% of 200?"

for model in models:
    start = time.time()
    result = enhance(query, model=model)
    elapsed = time.time() - start
    print(f"{model}: {elapsed:.2f}s")
    print(result[:100])
    print()

Query: What is 15% of 200?

Testing: tinyllama
💭 Reasoning:
1. Calculate the percentage value
2. Multiply the percentage value by the number
3. Get the final result

📝 Answer: To find 15% of 200, we'll follow the reasoning trace:

1. Calculate the percentage value: 
   To calculate a percentage, divide by 100.
   Percentage = 15 / 100 = 0.15

2. Multiply the percentage value by the number:
   Now multiply 0.15 by 200 to find 15% of 200.
   Result = 0.15 * 200 = 30

Therefore, 15% of 200 is 30.

✅ Confidence: 90%

⏱️  25.53s


SPEED RANKING
tinyllama            → 25.53s
💭 Reasoning:
1. Calculate the percentage value
2. Multiply the percentage value by the number
3. Get the final result

📝 Answer: To find 15% of 200, we'll follow the reasoning trace:

1. Calculate the percentage value: 
   To calculate a percentage, divide by 100.
   Percentage = 15 / 100 = 0.15

2. Multiply the percentage value by the number:
   Now multiply 0.15 by 200 to find 15% of 200.
   Result = 0.15 * 200 = 30

T

In [ ]:
# Example: Using a custom local model
# Just set provider="custom" and point to your model's OpenAI-compatible endpoint

from kaelum.core.config import LLMConfig, MCPConfig
from kaelum.runtime.orchestrator import MCP

# Option 1: Ollama (already working)
config_ollama = MCPConfig(
    llm=LLMConfig(
        provider="ollama",
        model="llama3.2:3b",
    )
)

# Option 2: vLLM (if you're running vLLM server)
config_vllm = MCPConfig(
    llm=LLMConfig(
        provider="vllm",
        model="your-model-name",
        base_url="http://localhost:8000/v1",
    )
)

# Option 3: YOUR custom model (NEW!)
# Works with any OpenAI-compatible server:
# - LiteLLM, LocalAI, FastChat, llama.cpp server, etc.
config_custom = MCPConfig(
    llm=LLMConfig(
        provider="custom",
        model="your-custom-model",
        base_url="http://localhost:5000/v1",  # Your model's endpoint
        api_key="optional",  # Only if your server requires it
    )
)

# Test with Ollama (change to config_custom when you have your model running)
mcp = MCP(config_ollama)
result = mcp.infer("What is 25% of 80?")

print("Query:", result["query"])
print("\nReasoning trace:")
for i, step in enumerate(result["trace"], 1):
    print(f"{i}. {step}")
print("\nFinal answer:", result["final"])

## Test Custom Local Model

KaelumAI supports using YOUR OWN open-source model without any external API calls.
Three main options:
1. **Ollama** - easiest (supports 100+ models)
2. **vLLM** - fastest (production-grade serving)
3. **Custom provider** - most flexible (any OpenAI-compatible server)